# Task 1 – Huffman Encoding and Information Representation

### Goal
Demonstrate understanding of variable-length coding and how information can be represented efficiently using Huffman coding.

## Step 1. Choose a Message
**Message chosen:** `PHOTOSYNTHESIS`

In [2]:
from collections import Counter

text = "PHOTOSYNTHESIS"
freq = Counter(text)
freq

Counter({'S': 3,
         'H': 2,
         'O': 2,
         'T': 2,
         'P': 1,
         'Y': 1,
         'N': 1,
         'E': 1,
         'I': 1})

## Step 2. Build Huffman Tree

In [3]:
import heapq
from collections import namedtuple

class Node(namedtuple("Node", ["char", "freq", "left", "right"])):
    def __lt__(self, other):
        if self.freq != other.freq:
            return self.freq < other.freq
        return (self.char or "") < (other.char or "")

def build_huffman_tree(text):
    freq = Counter(text)
    heap = [Node(ch, f, None, None) for ch, f in freq.items()]
    heapq.heapify(heap)
    while len(heap) > 1:
        left = heapq.heappop(heap)
        right = heapq.heappop(heap)
        new_node = Node(None, left.freq + right.freq, left, right)
        heapq.heappush(heap, new_node)
    return heap[0]

tree = build_huffman_tree(text)

## Step 3. Print Huffman Tree

In [4]:
def get_all_chars(node):
    if node is None:
        return []
    if node.char:
        return [node.char]
    return get_all_chars(node.left) + get_all_chars(node.right)

def node_label(node):
    if node.char:
        return f"{node.char}({node.freq})"
    else:
        chars = "".join(sorted(get_all_chars(node)))
        return f"[{chars}:{node.freq}]"

def get_levels(root):
    levels = []
    this_level = [root]
    while any(this_level):
        levels.append(this_level)
        next_level = []
        for node in this_level:
            if node:
                next_level.append(node.left)
                next_level.append(node.right)
            else:
                next_level.extend([None, None])
        this_level = next_level
    return levels

def print_tree(root):
    levels = get_levels(root)
    max_width = 80
    for i, level in enumerate(levels):
        per_level = max_width // (2 ** i)
        line = ""
        for node in level:
            label = node_label(node) if node else " "
            line += label.center(per_level)
        print(line.rstrip())
        print()

print("Huffman Tree (Centered Layout):")
print_tree(tree)

Huffman Tree (Centered Layout):
                                 [EHINOPSTY:14]

                [NPSY:6]                               [EHIOT:8]

      [NPY:3]               S(3)              [EHI:4]              [OT:4]

   Y(1)     [NP:2]                        [EI:2]     H(2)      O(2)      T(2)

           N(1) P(1)                     E(1) I(1)



## Step 4. Generate Huffman Codes and Encode Message

In [5]:
def get_codes(node, prefix="", code_map=None):
    if code_map is None:
        code_map = {}
    if node.char:
        code_map[node.char] = prefix or "0"
    else:
        get_codes(node.left, prefix + "0", code_map)
        get_codes(node.right, prefix + "1", code_map)
    return code_map

def huffman_encode(text, codes):
    return ''.join(codes[ch] for ch in text)

codes = get_codes(tree)
encoded = huffman_encode(text, codes)

print("Huffman Codes:", codes)
print("Encoded Message:", encoded)
print("Original length (ASCII):", len(text) * 8, "bits")
print("Encoded length (Huffman):", len(encoded), "bits")
print("Space saved:", len(text) * 8 - len(encoded), "bits")

Huffman Codes: {'Y': '000', 'N': '0010', 'P': '0011', 'S': '01', 'E': '1000', 'I': '1001', 'H': '101', 'O': '110', 'T': '111'}
Encoded Message: 0011101110111110010000010111101100001100101
Original length (ASCII): 112 bits
Encoded length (Huffman): 43 bits
Space saved: 69 bits


### Step 5. Reflection

Using Huffman coding, the word “PHOTOSYNTHESIS” is compressed from 112 bits (14 letters × 8 bits each) down to about 43 bits, saving approximately 62% of the original space. This happens because the letters that appear most often, like S, H, O, and T, are given shorter binary codes, while the less frequent ones use longer codes. The efficiency of Huffman coding really depends on how often each character appears — the more repetition, the greater the compression. However, if the receiver doesn’t use the same Huffman tree to decode the message, the bit sequence would be misinterpreted. Overall, Huffman coding works best when there’s a clear pattern or redundancy in the data.